In [1]:
import pickle

In [ ]:

with open('entrenamiento.p', 'rb') as file:
    training_data = pickle.load(file)
    

with open('prueba.p', 'rb') as file:
    test_data = pickle.load(file)
    

with open('validacion.p', 'rb') as file:
    validation_data = pickle.load(file)
